# 🌌 Análise de Dados Cósmicos

## Sistema Completo para Descoberta de Corpos Celestes e Análise de Vibrações Estelares

Este notebook demonstra:
- 🔭 **Detecção de corpos celestes** (planetas, asteroides, supernovas)
- ⭐ **Asterosismologia** - Análise de vibrações estelares
- 📡 **Busca por padrões e mensagens** em sinais cósmicos
- 📊 **Visualizações avançadas** dos resultados

---

## 1. Importar Bibliotecas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal, stats
import warnings
warnings.filterwarnings('ignore')

# Importar módulos do projeto
from celestial_detector import CelestialBodyDetector
from stellar_seismology import StellarSeismologyAnalyzer
from pattern_detector import PatternDetector
from visualizer import CosmicVisualizer

print("✓ Bibliotecas importadas com sucesso!")
print("="*60)

✓ Bibliotecas importadas com sucesso!


## 2. Gerar Dados de Exemplo

Vamos simular dados astronômicos para demonstração

In [2]:
def generate_planet_transit_data(n_points=5000, period=3.5, depth=0.01):
    """Gera curva de luz com trânsito planetário"""
    time = np.linspace(0, 30, n_points)  # 30 dias
    flux = np.ones(n_points)
    
    # Adicionar trânsito
    transit_duration = 0.1  # dias
    for cycle_start in np.arange(0, 30, period):
        mask = (time >= cycle_start) & (time < cycle_start + transit_duration)
        flux[mask] *= (1 - depth)
    
    # Adicionar ruído
    noise = np.random.normal(0, 0.001, n_points)
    flux += noise
    
    return time, flux

def generate_stellar_vibration_data(n_points=10000, nu_max=3000, delta_nu=135):
    """Gera dados de oscilações estelares"""
    time = np.linspace(0, 100, n_points)  # dias
    dt = time[1] - time[0]
    
    # Simular modos de oscilação
    flux = np.zeros(n_points)
    
    # Adicionar vários modos
    n_modes = 20
    for i in range(n_modes):
        freq = (nu_max + (i - n_modes//2) * delta_nu) * 1e-6  # Hz
        amplitude = np.random.uniform(0.0001, 0.001) * np.exp(-(i - n_modes//2)**2 / 50)
        phase = np.random.uniform(0, 2*np.pi)
        flux += amplitude * np.sin(2 * np.pi * freq * time * 86400 + phase)
    
    # Adicionar ruído
    noise = np.random.normal(0, 0.0005, n_points)
    flux += noise
    flux = flux / np.std(flux)  # Normalizar
    
    return time, flux

def generate_seti_signal(n_points=10000, has_pattern=True):
    """Gera sinal para análise SETI"""
    time = np.arange(n_points)
    
    if has_pattern:
        # Sinal com padrão - pulsos regulares modulados
        signal_data = np.random.normal(0, 1, n_points)
        
        # Adicionar pulsos periódicos
        pulse_period = 100
        for i in range(0, n_points, pulse_period):
            signal_data[i:i+10] += 5
        
        # Adicionar modulação em frequência específica
        carrier_freq = 0.05
        signal_data += 2 * np.sin(2 * np.pi * carrier_freq * time)
        
    else:
        # Sinal puramente aleatório
        signal_data = np.random.normal(0, 1, n_points)
    
    return signal_data

# Gerar dados
print("Gerando dados sintéticos...")
print("-"*60)

# 1. Dados de trânsito planetário
planet_time, planet_flux = generate_planet_transit_data()
print(f"✓ Curva de luz com trânsito: {len(planet_time)} pontos")

# 2. Dados de vibrações estelares
stellar_time, stellar_flux = generate_stellar_vibration_data()
print(f"✓ Dados de asterosismologia: {len(stellar_time)} pontos")

# 3. Sinal SETI com padrão
seti_signal = generate_seti_signal(has_pattern=True)
print(f"✓ Sinal SETI: {len(seti_signal)} pontos")

# 4. Adicionar evento transiente (supernova)
transient_time = np.copy(planet_time)
transient_flux = np.copy(planet_flux)
# Supernova em t=15 dias
event_start = 15
event_duration = 5
event_mask = (transient_time >= event_start) & (transient_time < event_start + event_duration)
# Curva de luz de supernova (subida rápida, queda lenta)
event_profile = np.linspace(0, 1, np.sum(event_mask)) ** 0.3
transient_flux[event_mask] += 0.3 * event_profile
print(f"✓ Evento transiente adicionado")

print("="*60)

Gerando dados sintéticos...
------------------------------------------------------------
✓ Curva de luz com trânsito: 5000 pontos
✓ Dados de asterosismologia: 10000 pontos
✓ Sinal SETI: 10000 pontos
✓ Evento transiente adicionado


## 3. 🔭 Detecção de Corpos Celestes

### Detectar planetas em trânsito

In [3]:
# Inicializar detector
detector = CelestialBodyDetector(sensitivity=3.0)

# Detectar planetas
print("🔍 Buscando exoplanetas...")
planets = detector.detect_transiting_planets(planet_time, planet_flux, min_period=1.0, max_period=10.0)

print(f"\n✓ Planetas detectados: {len(planets)}\n")
print("="*70)

for i, planet in enumerate(planets, 1):
    print(f"\n🪐 PLANETA CANDIDATO #{i}:")
    print(f"   Período orbital: {planet['period_days']:.3f} dias")
    print(f"   Profundidade do trânsito: {planet['transit_depth']*100:.4f}%")
    print(f"   Duração do trânsito: {planet['transit_duration_hours']:.2f} horas")
    print(f"   Confiança: {planet['confidence']:.1f}%")
    
    # Estimar tamanho do planeta (assumindo estrela tipo solar)
    radius_ratio = np.sqrt(planet['transit_depth'])
    planet_radius = radius_ratio * 1.0  # em raios solares
    planet_radius_earth = planet_radius * 109  # em raios terrestres
    print(f"   Raio estimado: {planet_radius_earth:.2f} R⊕")
    
print("\n" + "="*70)

🔍 Buscando exoplanetas...

✓ Planetas detectados: 0




### Detectar eventos transientes (supernovas, etc)

In [4]:
# Detectar eventos transientes
print("🔍 Buscando eventos transientes...")
transients = detector.detect_transient_events(transient_time, -2.5 * np.log10(transient_flux))

print(f"\n✓ Eventos transientes detectados: {len(transients)}\n")
print("="*70)

for i, event in enumerate(transients, 1):
    print(f"\n💥 EVENTO TRANSIENTE #{i}:")
    print(f"   Tipo: {event['type']}")
    print(f"   Tempo de pico: {event['peak_time']:.2f} dias")
    print(f"   Duração: {event['duration_days']:.2f} dias")
    print(f"   Amplitude: {event['amplitude']:.2f} magnitudes")
    print(f"   Tempo de subida: {event['rise_time']:.2f} dias")
    print(f"   Tempo de queda: {event['decay_time']:.2f} dias")

print("\n" + "="*70)

🔍 Buscando eventos transientes...

✓ Eventos transientes detectados: 1


💥 EVENTO TRANSIENTE #1:
   Tipo: Minor Transient
   Tempo de pico: 19.97 dias
   Duração: 1.60 dias
   Amplitude: 0.28 magnitudes
   Tempo de subida: 1.57 dias
   Tempo de queda: 0.03 dias



## 4. ⭐ Asterosismologia - Análise de Vibrações Estelares

In [5]:
# Inicializar analisador de asterosismologia
seismo = StellarSeismologyAnalyzer()

print("🎵 Analisando vibrações estelares...")
print("="*70)

# Analisar oscilações
seismo_analysis = seismo.analyze_stellar_vibrations(stellar_time, stellar_flux, cadence=30.0)

# Gerar relatório
report = seismo.generate_seismology_report(seismo_analysis)
print(report)

🎵 Analisando vibrações estelares...
RELATÓRIO DE ASTEROSISMOLOGIA - ANÁLISE DE VIBRAÇÕES ESTELARES

📊 FREQUÊNCIAS DE OSCILAÇÃO:
----------------------------------------------------------------------
  ν_max (freq. potência máxima): 176.50 μHz
  Δν (grande separação):         5.32 μHz
  Largura do envelope:           121.53 μHz

⭐ PARÂMETROS ESTELARES DERIVADOS:
----------------------------------------------------------------------
  Massa:                77.26 M☉
  Raio:                 36.78 R☉
  log g:                3.19
  Densidade:            0.00 ρ☉
  Temperatura efetiva:  2863 K
  Idade estimada:       0.06 Gyr
  Estágio evolutivo:    Red Giant Branch

🎵 MODOS DE OSCILAÇÃO DETECTADOS: 11
----------------------------------------------------------------------
  Modo 1: 156.13 μHz - Quadrupole (l=2)
  Modo 2: 161.34 μHz - Quadrupole (l=2)
  Modo 3: 172.22 μHz - Quadrupole (l=2)
  Modo 4: 174.31 μHz - Dipole (l=1)
  Modo 5: 176.62 μHz - Radial (l=0)
  Modo 6: 182.64 μHz - Quadrupole

## 5. 📡 Detecção de Padrões e Mensagens (SETI)

In [ ]:
# Inicializar detector de padrões
pattern_detector = PatternDetector(significance_level=0.001)

print("📡 Analisando sinal para padrões não-aleatórios...")
print("="*70)

# Analisar sinal
pattern_analysis = pattern_detector.analyze_signal(seti_signal, sample_rate=1.0)

# Gerar relatório
pattern_report = pattern_detector.generate_pattern_report(pattern_analysis)
print(pattern_report)

📡 Analisando sinal para padrões não-aleatórios...


### Comparar com sinal puramente aleatório

In [ ]:
# Gerar e analisar sinal aleatório para comparação
random_signal = generate_seti_signal(n_points=10000, has_pattern=False)
random_analysis = pattern_detector.analyze_signal(random_signal, sample_rate=1.0)

print("📊 COMPARAÇÃO DE SCORES:")
print("="*70)
print(f"\nSinal com Padrão:     Score = {pattern_analysis['artificiality_score']['score']}/100")
print(f"  Classificação: {pattern_analysis['artificiality_score']['classification']}")
print(f"\nSinal Aleatório:      Score = {random_analysis['artificiality_score']['score']}/100")
print(f"  Classificação: {random_analysis['artificiality_score']['classification']}")
print("\n" + "="*70)

## 6. 📊 Visualizações

In [ ]:
# Inicializar visualizador
viz = CosmicVisualizer(figsize=(15, 10))

# Configurar matplotlib para exibir inline
%matplotlib inline
plt.style.use('dark_background')

### Visualizar detecções de corpos celestes

In [ ]:
viz.plot_celestial_detections(transient_time, transient_flux, planets, transients)

### Visualizar asterosismologia

In [ ]:
viz.plot_stellar_seismology(seismo_analysis)

### Visualizar análise de padrões

In [ ]:
viz.plot_pattern_analysis(seti_signal, pattern_analysis, sample_rate=1.0)

### Dashboard resumido de todas as análises

In [ ]:
# Criar dashboard com todos os resultados
celestial_results = {
    'planets': planets,
    'transients': transients
}

viz.plot_summary_dashboard(
    celestial_results=celestial_results,
    seismology_results=seismo_analysis,
    pattern_results=pattern_analysis
)

## 7. 🎓 Experimento: Análise com Dados Reais

Para usar dados reais do Kepler ou TESS, você pode integrar com `lightkurve`:

In [ ]:
# Exemplo de integração com dados reais (necessita lightkurve)
"""
import lightkurve as lk

# Baixar dados do Kepler para uma estrela específica
search_result = lk.search_lightcurve('KIC 11904151', author='Kepler', cadence='short')
lc = search_result.download_all()

# Preparar dados
time = lc.time.value
flux = lc.flux.value

# Analisar
detector = CelestialBodyDetector()
planets = detector.detect_transiting_planets(time, flux)

# Asterosismologia
seismo = StellarSeismologyAnalyzer()
seismo_results = seismo.analyze_stellar_vibrations(time, flux)
"""

print("💡 DICA: Descomente o código acima e instale 'lightkurve' para usar dados reais!")
print("   pip install lightkurve")

## 8. 📝 Conclusões

### O que este sistema pode fazer:

✅ **Detecção de Exoplanetas**
- Identifica trânsitos planetários em curvas de luz
- Calcula período orbital, profundidade do trânsito e tamanho do planeta
- Avalia confiança da detecção

✅ **Asterosismologia (Vibrações Estelares)**
- Analisa oscilações estelares para determinar propriedades internas
- Calcula massa, raio, idade e gravidade da estrela
- Identifica modos de pulsação (radial, dipolar, quadrupolar)
- Detecta rotação estelar

✅ **Detecção de Eventos Transientes**
- Identifica supernovas, novas, flares estelares
- Classifica tipo de evento baseado em amplitude e duração
- Mede tempos de subida e queda

✅ **Busca por Padrões/Mensagens (SETI)**
- Testa aleatoriedade do sinal
- Detecta periodicidades e pulsos regulares
- Busca padrões matemáticos (números primos, razões especiais)
- Analisa entropia e modulação
- Calcula "score de artificialidade"

### Aplicações:

🔬 **Pesquisa Científica**
- Descoberta de exoplanetas
- Caracterização de estrelas
- Monitoramento de eventos transientes

🛰️ **SETI e Astrobiologia**
- Busca por sinais de inteligência extraterrestre
- Análise de padrões não-naturais

📚 **Educação**
- Demonstração de técnicas de análise de dados astronômicos
- Visualização de fenômenos cósmicos

---

### Próximos passos:

1. Testar com dados reais do Kepler/TESS
2. Implementar machine learning para classificação
3. Adicionar análise espectroscópica
4. Criar interface web interativa